In [ ]:
import os
import sys
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from matplotlib.patches import Rectangle

In [ ]:
sns.set_style({'axes.axisbelow': True,
               'axes.edgecolor': '.8',
               'axes.facecolor': 'white',
               'axes.grid': True,
               'axes.labelcolor': '.15',
               'axes.spines.bottom': True,
               'axes.spines.left': True,
               'axes.spines.right': True,
               'axes.spines.top': True,
               'figure.facecolor': 'white',
               'font.family': ['sans-serif'],
               'font.sans-serif': ['Arial',
                'DejaVu Sans',
                'Liberation Sans',
                'Bitstream Vera Sans',
                'sans-serif'],
               'grid.color': '.8',
               'grid.linestyle': '--',
               'image.cmap': 'rocket',
               'lines.solid_capstyle': 'round',
               'patch.edgecolor': 'w',
               'patch.force_edgecolor': True,
               'text.color': '.15',
               'xtick.bottom': True,
               'xtick.color': '.15',
               'xtick.direction': 'in',
               'xtick.top': True,
               'ytick.color': '.15',
               'ytick.direction': 'in',
               'ytick.left': True,
               'ytick.right': True})

In [ ]:
figsave_out = '../Documentation/report/src/images/'
figsave_format = 'pdf'
figsave_dpi = 200

# Bold text format
b1 = '\033[1m'
b0 = '\033[0m'

In [ ]:
with open('../data/spectra.dat') as f:
    lines = [line.split('\n')[0] for line in f]
    data_bin = [int(line) \
                for line in lines[lines.index('<<DATA>>')+1:lines.index('<<END>>')]]
    data_roi = sorted([[int(i) for i in line.split(' ')] \
                       for line in lines[lines.index('<<ROI>>')+1:lines.index('<<DATA>>')]])

# Convert ROI to keV from bin values
data_roi_e = [[0.9255 * x - 1.7343 for x in i] for i in data_roi]

data_roi_y = [[0, 3500],
              [0, 2000],
              [0, 15000],
              [0, 2000],
              [0, 700],
              [0, 1000]]

In [ ]:
# There should be 4095 bins in the amplitude-analizator, but only 4083 did actually work
# Here : len(data_bin) == 4083
amp_an_bins = len(data_bin)
# Equation for transforming from bins to keV
# keV = 0.9255 * BIN_VALUE - 1.7343
LOWER_LIM = 0.9255 * 0 - 1.7343
UPPER_LIM = 0.9255 * amp_an_bins - 1.7343
print(b1 + 'Lower limit of X axis :' + b0, LOWER_LIM)
print(b1 + 'Upper limit of X axis :' + b0, UPPER_LIM)
# Bins with energy values
energy_bins = np.linspace(LOWER_LIM, UPPER_LIM, amp_an_bins)

In [ ]:
# Gamma photon names
# 143.768 keV : U235 - g4,1 (Here: 145.28 keV)
# 163.358 keV : U235 - g5,1 (Here: 164.75 keV)
# 185.722 keV : U235 - g4,0 (Here: 186.79 keV)
# 205.316 keV : U235 - g5,0 (Here: 206.10 keV)
# 766.708 keV : Pa234m - g4,1 (Here: 764.24 keV)
# 1001.441 keV : Pa234m - g9,1 (Here: 998.77 keV)
gammas = ['$^{235}$U - $\gamma_{4,1}$',
          '$^{235}$U - $\gamma_{5,1}$',
          '$^{235}$U - $\gamma_{4,0}$',
          '$^{235}$U - $\gamma_{5,0}$',
          '$^{234}$Pa$^{m}$ - $\gamma_{4,1}$',
          '$^{234}$Pa$^{m}$ - $\gamma_{9,1}$']

gamma_names = ['U235_g4_1',
               'U235_g5_1',
               'U235_g4_0',
               'U235_g5_0',
               'Pam234_g4_1',
               'Pam234_g9_1']

gamma_colors = ['tab:red',
                'tab:green',
                'tab:orange',
                'tab:purple',
                'tab:grey',
                'tab:brown',]

## Plot spectra and characteristic peaks peaks

In [ ]:
def plot_spectra(lims=None, save=False, plot_type='line', name=None):
    nrows = 1
    ncols = 2
    fig = plt.figure(figsize=(ncols*14,nrows*12))
    G = gridspec.GridSpec(nrows, ncols)
    G.update(hspace=0.35, wspace=0.2)
    axes = [plt.subplot(G[i]) for i in range(ncols)]

    suptitlesize = 50
    titlesize = 40
    axislabelsize = 40
    axisticksize = 30
    legendsize = 25

    if lims is not None:
        linewdt = 2
        '''
        if name is not None:
            plt.suptitle('{0} gamma peak of my sample '.format(name) +
                         '(small, yellow crystal)\n',
                         fontweight='bold', fontsize=suptitlesize, y=1.08)

        else:
            plt.suptitle('One of the gamma peak of my sample ' +
                         '(small, yellow crystal)\n',
                         fontweight='bold', fontsize=suptitlesize, y=1.08)
        '''
    else:
        linewdt = 2
        '''
        plt.suptitle('Gamma spectra of my measured sample ' +
                     '(small, yellow crystal)',
                     fontweight='bold', fontsize=suptitlesize, y=1.08)
        '''

    # Linear scale
    if plot_type == 'line':
        axes[0].plot(energy_bins, data_bin, lw=linewdt)
        axes[0].fill_between(x=energy_bins, y1=data_bin)
        for idx, roi in enumerate(data_roi_e):
            axes[0].axvline(x=np.mean(roi), label=gammas[idx],
                            color=gamma_colors[idx], ls='--', alpha=1,
                            zorder=0)
        #axes[0].set_xlim(-50,1100)
        axes[0].legend(loc='upper center', fontsize=legendsize)

    elif plot_type == 'hist':
        axes[0].bar(energy_bins, data_bin,
                    width=0.9, align='center')
    axes[0].set_title('Linear scale', fontsize=titlesize, y=1.02)

    # Log scale
    if plot_type == 'line':
        axes[1].plot(energy_bins, data_bin, lw=linewdt)
        axes[1].fill_between(x=energy_bins, y1=data_bin)
        for idx, roi in enumerate(data_roi_e):
            axes[1].axvline(x=np.mean(roi), label=gammas[idx],
                            color=gamma_colors[idx], ls='--', alpha=1,
                            zorder=0)
            
        #axes[1].set_xlim(-50,1100)
        axes[1].legend(loc='upper center', fontsize=legendsize)

    elif plot_type == 'hist':
        axes[1].bar(energy_bins, data_bin,
                    width=0.9, align='center')

    axes[1].set_yscale('log')
    axes[1].set_title('Logarithmic scale', fontsize=titlesize, y=1.02)

    for i in range(ncols):
        axes[i].set_xlabel('Energy [keV]', fontsize=axislabelsize)
        axes[i].set_ylabel('Intensity [n]', fontsize=axislabelsize)
        axes[i].tick_params(axis='both', which='major', labelsize=axisticksize, pad=15)
        if lims is not None:
            assert(np.shape(lims) == (2, 2)), "\'lims\' should be a (2,2) shape array!"
            axes[i].set_xlim(lims[0][0], lims[0][1])
            axes[0].set_ylim(lims[1][0], lims[1][1])
            axes[1].set_ylim(1, lims[1][1])

    if save:
        if lims is not None:
            if name is not None:
                plt.savefig((figsave_out + 'spectra_lims_{0:.2f}_{1:.2f}_{2}.'.format(lims[0][0], lims[0][1], name)) +
                             figsave_format,
                            format=figsave_format,
                            dpi=figsave_dpi,
                            bbox_inches='tight')
            else:
                plt.savefig((figsave_out + 'spectra_lims_{0:.2f}_{1:.2f}.'.format(lims[0][0], lims[0][1])) +
                             figsave_format,
                            format=figsave_format,
                            dpi=figsave_dpi,
                            bbox_inches='tight')

        else:
             plt.savefig(figsave_out + 'full_spectra.' +
                         figsave_format,
                         format=figsave_format,
                         dpi=figsave_dpi,
                         bbox_inches='tight')   

    plt.show()

In [ ]:
lim = False

if lim:
    ymin_lim = 0 - np.max(data_bin)*(1/15)
    ymax_lim = np.max(data_bin)*(1 + 1/15)
    plot_spectra(lims=([-50,1100], [ymin_lim, ymax_lim]), save=True, plot_type='line')
else:
    plot_spectra(lims=None, save=True, plot_type='line')

In [ ]:
for i in range(1, len(data_roi_e)):
    plot_spectra(lims=(data_roi_e[i], data_roi_y[i]), save=True, plot_type='hist', name=gamma_names[i])

## Calculate activity of monitored radioactive nucleii

$$
A
=
\frac{S}{t \cdot I \left( E, n \right) \cdot \eta \left( E, n \right)}
$$

In [ ]:
t = 1073
S = np.array([3375, 1976, 20836, 1666, 914, 1550])
deltaS = np.array([2.35, 4.05, 0.73, 2.86, 3.75, 2.16]) / 100
DeltaS = deltaS * S
eta = np.array([1330.59e-4, 1232.67e-4, 1065.56e-4, 1008.12e-4, 270.98e-4, 212.55e-4])
deltaeta = np.array([2.218, 2.336, 2.129, 3.085, 2.491, 2.365]) / 100
Deltaeta = deta * eta
I = np.array([13.20, 5.855, 63.41, 5.465, 0.329, 0.856]) / 100
lambda_i = [3.12e-17, 3.12e-17, 3.12e-17, 3.12e-17, 4.92e-18, 4.92e-18]

In [ ]:
A = S / (t * eta * I)
deltaA = np.sqrt(deltaS**2 + deltaeta**2)
DeltaA = deltaA * A

In [ ]:
print(b1 + 'Activities of the measured elements:' + b0)
for idx, act in enumerate(A):
    print('{0} : ({1:.1f} +- {2:.1f}) Bq'.format(gamma_names[idx], act, DeltaA[idx]))

In [ ]:
N = A / lambda_i
deltaN = deltaA
DeltaN = deltaN * N

In [ ]:
print(b1 + 'Particle numbers of the measured elements:' + b0)
for idx, n in enumerate(N):
    print('{0} : ({1:.2e} +- {2:.2e})'.format(gamma_names[idx], n, DeltaN[idx]))

In [ ]:
N_235 = np.mean(N[:4])
deltaN_235 = np.sqrt(np.square(deltaN[:4]).sum())
DeltaN_235 = deltaN_235 * N_235
N_238 = np.mean(N[4:])
deltaN_238 = np.sqrt(np.square(deltaN[4:]).sum())
DeltaN_238 = deltaN_238 * N_238

In [ ]:
print(b1 + 'Number of U235 particles : ' + b0 + '{0:.2e} +- {1:.2e}'.format(N_235, DeltaN_235))
print(b1 + 'Number of U238 particles : ' + b0 + '{0:.2e} +- {1:.2e}'.format(N_238, DeltaN_238))

In [ ]:
C_235 = N_235 / (N_235 + N_238)
C_238 = N_238 / (N_235 + N_238)
deltaC_235 = deltaN_235
deltaC_238 = deltaN_238
DeltaC_235 = deltaC_235 * C_235
DeltaC_238 = deltaC_238 * C_238

In [ ]:
print(b1 + 'Relative proportion of U235 : ' + b0 + '({0:.2f} +- {1:.2f}) %'.format(C_235 * 100, DeltaC_235 * 100))
print(b1 + 'Relative proportion of U238 : ' + b0 + '({0:.2f} +- {1:.2f}) %'.format(C_238 * 100, DeltaC_238 * 100))

In [ ]:
A_tot = ((A / np.square(DeltaA)).sum())/((1 / np.square(DeltaA)).sum())
DeltaA_tot = 1 / np.sqrt((1 / np.square(DeltaA)).sum())

In [ ]:
print(b1 + 'Total activity of the sample : ' + b0 + '({0:.2f} +- {1:.2f})'.format(A_tot, DeltaA_tot))